# SIXT33N version B
## Phase 2: Principal Component Analysis

### EE 16B: Designing Information Devices and Systems II, Fall 2015

**Name 1**:

**Login**: ee16b-


**Name 2**:

**Login**: ee16b-

##Table of Contents

* [Introduction](#intro)
* [Part 1: Data Collection](#part1)
* [Part 2: Principal Component Analysis](#part2)
* [Part 3: Classification](#part3)
* [Part 4: Launchpad Implementation](#part4)

<a id='intro'></a>
## Introduction

In this phase we will process the ADC readings into some form of data that is suitable for PCA. 

When you think of the speech signals you saw on your scope, you will notice that the shape of the speech wave was the most distinctive part of each word. Taking just the shape of the magnitude of a signal is called enveloping. So, we want to do some filtering to retrieve the envelope of the audio signal. We can then train the PCA off of just this envelope and build a classifier to classify new data points. Finally, we will implement the classifier on the Launchpad.

The goals of this phase are as follows:
- Generate envelope, threshold to get snippets
- PCA + Classifier (2-3 commands)
- Check accuracy
- PCA projection on Launchpad

<a id='part1'></a>
##<span style="color:blue">Part 1: Data Collection</span>

###Materials
- Microphone front-end circuit
- Launchpad + USB

Firstly, set your circuit and your Launchpad up just like in the last phase. Do not forget to power the first op-amp with the voltage regulator and the second op-amp from the Launchpad 3.3V pin. 

To collect speech data, upload the provided sketch <b>`collect-data-envelope.ino`</b> to your Launchpad. This sketch will turn on the red Launchpad LED to show that it is recording. The Launchpad will record 2 seconds of audio at a time, sampled every 0.35ms. When the red LED lights up, say the word you want to record before it turns off. To make your life easier, pronounce the words consistently. Try a few different words that you think will classify well. The Launchpad will apply an enveloping function (discussed later) which reduces the data from several thousand samples to 172 samples. 

To record the data in the PC, run:

<b>`python collect-data-envelope.py log.csv`</b>

This will append the data streamed back by the Launchpad to `log.csv`. Collect data for at least 3 words that have something to do with speeding up or slowing down. You might want to look at your breadboard output on the scope while you collect. Make sure that your commands are not too loud and getting clipped, nor that they are too soft and hard to pick out from noise. 

We recommend using words that have either distinct intonation or words that have different lengths. For each word, collect at least 16 samples and save them as separate csv files.

<a id='part2'></a>
##<span style="color:blue">Part 2: Principal Component Analysis</span>

Before we can use the recorded data for PCA, we must first process the data.

We have tried several methods for you and found that enveloping the recorded audio works well for PCA since it only retains the general shape of the sound, independent of the actual frequency or pitch of the voice. You already used the implemented enveloping function in the sketch <b>`collect-data-envelope.ino`</b> from part 1. The normalized input and output of the function looks like the plot below.

<center>
<img width="500px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/proj-envelope.png">
</center>

It is not necessary for you to understand the enveloping function well enough to implement it (since we have already done it for you), but just in case you are curious the enveloping function is described in the following pseudocode:

<code><b>Enveloping function</b>
Divide the whole signal to a block of 16 samples
For each chunk:
    Find the mean of the chunk
    Subtract each sample by the mean
    Find the sum of the absolute value of each sample
</code>

Now that we have the envelope of the waveform, we can start developing our PCA classifier. First, load the recorded data from the csv files.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.io
from scipy import interpolate
import csv
%matplotlib inline

In [ ]:
def read_csv(filename):
    data = []
    with open(filename, 'r') as csvfile:
        r = csv.reader(csvfile, delimiter=' ')
        for row in r:
            data.append([float(i) for i in row[0].split(',')])
    return np.array(data)

In [ ]:
# Load data from csv
# YOUR CODE HERE
word1 = 
word2 =
#Optional: word3 =

# Take the same number of readings for both genres to be fair
num_samples = min(np.shape(word1)[0], np.shape(word2)[0])
word1 = word1[:num_samples,:]
word2 = word2[:num_samples,:]

Plot your data and get a feel for how it looks enveloped.

In [ ]:
# Plot all word1 samples
plt.plot(word1.T)
plt.show()

As you can see above, the speech is only a small part of the 2 second window, and it starts at different times. PCA is not good at interpreting delay, so we need to somehow start in the same place each time and capture a smaller segment of the 2 second sample where the speech is present. To do this, we will use a thresholding algorithm just like the one used in the BMI labs. First, we define a threshold relative to the maximum value of the data. We say that any signal that crosses the threshold is the start of a speech command. In order to not lose the first couple samples of the speech command, we say that the command starts `pre_length` samples before the threshold is crossed. We then take a window of the data that is `length` long, and try to capture the entire sound of the command in that window.

Play around with the parameters `length`, `pre_length` and `threshold` in the cells below to find appropriate values corresponding to your voice and chosen commands. You should see the results and how much of your command you captured in the plots generated below. When you are satisfied, note down the values of `length`, `pre_length` and `threshold` - <b>you will need to add them to the Launchpad sketch later.</b>

In [ ]:
def get_snippets(data, length, pre_length, thres):
    data_out = np.zeros((np.shape(data)[0], length))
    
    for rnum, row in enumerate(data):
        # Find the threshold
        row_thres = thres*np.max(row)

        # Figure out when interesting snippet starts
        block = pre_length
        while (row[block] < row_thres):
            block = block + 1
        block = min(block, 172 - length)
        data_out[rnum,:] = row[block-pre_length:block-pre_length+length]
        
        # Normalization
        data_out[rnum,:] = data_out[rnum,:] / np.sum(data_out[rnum,:])
        
    return data_out

In [ ]:
length = 40        # Adjust this
pre_length = 5     # Adjust this
threshold = 0.5    # Adjust this

word1_snippets = get_snippets(word1, length, pre_length, threshold)
plt.plot(word1_snippets.T)
plt.show()
plt.figure()
word2_snippets = get_snippets(word2, length, pre_length, threshold)
plt.plot(word2_snippets.T)
plt.show()

Now that we have our data in a nice format, we can build the PCA input matrix from that data. The function `np.vstack` might be helpful here. Next, take the SVD to find the principal components and plot the sigma values.

In [ ]:
# YOUR CODE HERE #
A = 

# Zero-mean the matrix A. Note down the mean for the Launchpad code
# YOUR CODE HERE #
A_means = 
A = 

In [ ]:
# Take the SVD of matrix A
# YOUR CODE HERE #
[u,s,v] = 

In [ ]:
# Plot out the sigma values
plt.stem(s)
plt.xlim([-1,10])

**<span style="color:red">How many principal components do you need?</span>** 

YOUR ANSWER HERE

In [ ]:
# Plot the principal component(s)
# YOUR CODE HERE #
plt.plot(...)

Now project the data in the matrix A onto the new basis and plot it. Do you see clustering? Do you think you can separate the data easily? If not, you might need to try new words.

In [ ]:
# Project the data onto the new basis
# YOUR CODE HERE #
proj = 

plt.scatter(np.linspace(0,num_samples,num_samples), proj[0:num_samples,0], c=['blue'], edgecolor='none')
plt.scatter(np.linspace(num_samples,num_samples*2,num_samples), proj[num_samples:num_samples*2,0], c=['red'], edgecolor='none')\
plt.legend(['word1', 'word2'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

<a id='part3'></a>
##<span style="color:blue">Part 3: Classification</span>

Using the plot above, we will define a way of classifying the different words. Fill in the skeleton code below to classify a vector in your new basis. 

Use a few of speech samples you collected at the beginning and test your classification on them. Don't forget to do the same processing on these samples that you trained with (i.e. do zero-mean by subtracting the mean of the original matrix A).

You need to be a little careful when creating the classifying algorithm since we do not want SIXT33N to pick up random sounds and treat them as one of the commands. To do this, pick a reasonably tight boundary for your classification.

Note the colors for each word in the legend of the plots above.

In [ ]:
def classify(vector, new_basis):
    """
    Classifies a new reading vector into a word.
    Inputs:
        vector: new data point vector
        new_basis: matrix or vector with the basis of the new space
    Output:
        String of the classified word
    """
    # YOUR CODE HERE
    proj = 
    if (...):
        return '...'
    if (...):
        return ''...'

In [ ]:
# Try out the classification function
print(classify(A[0,:], ...)) # Modify to use other vectors

<a id='part4'></a>
##<span style="color:blue">Part 4: Launchpad Implementation</span>

###Materials
- Microphone front-end circuit
- Launchpad + USB

We have implemented most of the algorithms you worked on above. Quickly glance through <b>`classify.ino`</b>. It contains the same base code as the <b>`collect-data-envelope.ino`</b> from before, with some extra functionality. 

First, we modified the enveloping function to also include threshold detection. The function now writes the normalized segment of interest (where the speech is) to an array of 172 floats. The function requires the constants <b>`SNIPPET_SIZE`</b>, <b>`PRESNIPPET`</b> and <b>`THRESHOLD`</b>, so fill in the `# YOUR CODE HERE #` lines in the top of the sketch where these constants are defined.

Second, the main loop now also performs PCA projection on the normalized segment using the vectors <b>`pca_vec`</b> and <b>`pca_mean`</b>. Fill these arrays up in the top of the sketch. The sketch assumes you are only using 1 principal component. If you need more than 1 principal components, modify the sketch to reflect that by creating a second dimension of `pca_vec`. An example of filling up an array is shown below for syntax.

<code>float pca_vec[3] = {0.1234, 0.5678, 0.9012};</code>

Lastly, it contains some skeleton code that classifies the projected value. Modify lines 95-98 to mimic your `classify` function from Part 3 to implement the classifier. The Launchpad should print out the words it recognizes. This will later be used as an input to the motor controller.